Evaluation Space

In [26]:
import numpy as np
import pandas as pd
import sys
import os
import time
from sqlfluff.core import Linter
import re
from difflib import SequenceMatcher
from sqlrespcompare import run_compare_query_results

In [9]:
print(sys.path)

['/usr/lib/python311.zip', '/usr/lib/python3.11', '/usr/lib/python3.11/lib-dynload', '', '/home/kengqiangmk/Projects/chat-to-database-chatbot/venv/lib/python3.11/site-packages', '/home/kengqiangmk/Projects/chat-to-database-chatbot/chat2dbchatbot']


In [3]:


# # Get the absolute path of the tools folder
project_dir = os.path.abspath("..")  # Assuming the eval folder is in the project folder
chat2dbchatbot_dir = os.path.join(project_dir, "chat2dbchatbot")

# # Add the tools directory to the system path
sys.path.append(chat2dbchatbot_dir)


In [10]:
from tools.tagsql import run_tag_pipeline
# from tools.rag import run_rag_pipeline

In [16]:
eval_file_path = './data/custom_db_exp.csv'
df = pd.read_csv(eval_file_path)
df.head()

,sql_complexity,sql_complexity_description,sql_prompt,sql
0,basic SQL,basic SQL with a simple select statement,Show all invoices that were made during Christ...,SELECT * FROM invoice WHERE (TO_CHAR(invoice_d...
1,single join,"only one join (specify inner, outer, cross)",Find the top 3 artists by total albums,"SELECT AR.name, COUNT(AR.name) as TotalAlbums ..."
2,aggregation and single join,"aggregation functions (COUNT, SUM, AVG, MIN, M...",list classic composers and their corresponding...,"SELECT T.composer, AVG(T.milliseconds) from tr..."
3,basic SQL,basic SQL with a simple select statement,list all the countries our customers are from,SELECT distinct country from customer;
4,simple aggregation,basic SQL with a simple select statement,What is the total invoice amount by country?,"SELECT billing_country, SUM(total) from invoic..."


In [31]:

linter = Linter(dialect='postgres')
lint_result = ""
for i in range(len(df)):
    start_time = time.perf_counter()
    gen_sql = await run_tag_pipeline(df.iloc[i]['sql_prompt'], "OpenAI", 0.1, "query_synthesis")
    end_time = time.perf_counter()
    latency = (end_time - start_time)*1000 # convert seconds to milli seconds
    lint_return = linter.lint_string(gen_sql)
    if lint_return.violations:
        for violation in lint_return.violations:
            lint_result += f"- {violation.description} (Line: {violation.line_no}, Pos: {violation.line_pos})\n"
    else: 
        lint_result = "no error"
    compact_gen_sql = re.sub(r"\s+", "", gen_sql).lower()
    compact_sql = re.sub(r"\s+", "", df.iloc[i]['sql']).lower()
    print(compact_gen_sql)
    print(compact_sql)
    df.loc[i, 'exp_gen_similarity'] = SequenceMatcher(None, compact_sql, compact_gen_sql).ratio()
    df.loc[i, 'exp_gen_len_diff'] = len(compact_gen_sql) - len(compact_sql)
    df.loc[i, 'is_exp_gen_resp_equal'] = run_compare_query_results(compact_gen_sql, compact_sql)
    df.loc[i, 'gen_sql'] = gen_sql
    df.loc[i, 'latency'] = latency
    df.loc[i, 'lint_result'] = lint_result

df.head(2)


selectinvoice_id,customer_id,invoice_date,totalfrominvoicewhereinvoice_date::datein('2021-12-24','2021-12-25','2021-12-26','2021-12-31','2022-01-01');
select*frominvoicewhere(to_char(invoice_date,'mm-dd')in('12-24','12-25','12-26','12-31','01-01'));
selectartist.name,count(album.album_id)astotal_albumsfromartistjoinalbumonartist.artist_id=album.artist_idgroupbyartist.nameorderbytotal_albumsdesclimit3;
selectar.name,count(ar.name)astotalalbumsfromalbumalinnerjoinartistaronal.artist_id=ar.artist_idgroupbyar.nameorderbytotalalbumsdesclimit3;
selectcomposer,avg(milliseconds)fromtrackinnerjoingenreontrack.genre_id=genre.genre_idwheregenre.name='classic'groupbycomposer;
selectt.composer,avg(t.milliseconds)fromtracktleftjoingenregont.genre_id=g.genre_idwhereg.namelike'%classic%'groupbyt.composer;
selectdistinctcountryfromcustomer
selectdistinctcountryfromcustomer;
selectbilling_country,sum(total)astotal_invoice_amountfrominvoicegroupbybilling_countryorderbytotal_invoice_amountdesc;
selectbill

,sql_complexity,sql_complexity_description,sql_prompt,sql,gen_sql,latency,lint_result,exp_gen_similarity,exp_gen_len_diff,is_exp_gen_resp_equal
0,basic SQL,basic SQL with a simple select statement,Show all invoices that were made during Christ...,SELECT * FROM invoice WHERE (TO_CHAR(invoice_d...,"SELECT invoice_id, customer_id, invoice_date, ...",18419.253565,- Select targets should be on a new line unles...,0.637097,52.0,False
1,single join,"only one join (specify inner, outer, cross)",Find the top 3 artists by total albums,"SELECT AR.name, COUNT(AR.name) as TotalAlbums ...","SELECT artist.name, COUNT(album.album_id) AS t...",8342.962453,- Select targets should be on a new line unles...,0.819113,15.0,False


In [28]:
print(df.iloc[0]['gen_sql'])
print(df.iloc[0]['lint_result'])
print(df['is_exp_gen_resp_equal'].tolist())

SELECT invoice_id, customer_id, invoice_date, total 
FROM invoice 
WHERE invoice_date::date IN ('2022-12-24', '2022-12-25', '2022-12-26', '2022-12-31', '2023-01-01')
ORDER BY invoice_date;
- Select targets should be on a new line unless there is only one select target. (Line: 1, Pos: 1)
- Unnecessary trailing whitespace. (Line: 1, Pos: 52)
- Unnecessary trailing whitespace. (Line: 2, Pos: 13)
- Line is too long (98 > 80). (Line: 3, Pos: 1)
- Files must end with a single trailing newline. (Line: 4, Pos: 22)

[False, False, False, False, False, False, False, False, False]


In [ ]:
print(df.iloc[3]['gen_sql'])
print(df.iloc[3]['lint_result'])